In [8]:
import numpy as np
from filterpy.kalman import KalmanFilter

# Orientation estimation using extended kalman filter
  Based on: X. Yun and E. R. Bachmann, ‘Design, Implementation, and Experimental Results of a Quaternion-Based Kalman Filter for Human Body Motion Tracking’, IEEE Trans. Robot., vol. 22, no. 6, pp. 1216–1227, Dec. 2006, doi: 10.1109/TRO.2006.886270.

In [5]:
# this is the Jacobian (linearized version) of the state transition, equation (13)
def phi(x, Ts, tau1, tau2, tau3):
    return np.array([[np.exp(-Ts/tau1), 0, 0, 0, 0, 0, 0],
                    [0, np.exp(-Ts/tau2), 0, 0, 0, 0, 0],
                    [0, 0, np.exp(-Ts/tau3), 0, 0, 0, 0],
                    [-(x[4]*Ts)/2, -(x[5]*Ts)/2, -(x[6]*Ts)/2, 1, -(x[0]*Ts)/2, -(x[1]*Ts)/2, -(x[2]*Ts)/2],
                    [(x[3]*Ts)/2, -(x[6]*Ts)/2, (x[5]*Ts)/2, (x[0]*Ts)/2, 1, (x[2]*Ts)/2, -(x[1]*Ts)/2],
                    [(x[6]*Ts)/2, (x[3]*Ts)/2, -(x[4]*Ts)/2, (x[1]*Ts)/2, -(x[2]*Ts)/2, 1, (x[0]*Ts)/2],
                    [-(x[5]*Ts)/2, (x[4]*Ts)/2, (x[3]*Ts)/2, (x[2]*Ts)/2, (x[1]*Ts)/2, -(x[0]*Ts)/2, 1]])

In [9]:
kf = KalmanFilter(dim_x=7, dim_z=7)
kf.x = np.array([0., 0., 0., 0., 0., 0., 0.])
Ts = 0.04  # 25Hz samples

kf.H = np.eye(7)

tau1 = 0.5
tau2 = 0.5
tau3 = 0.5
D = 0.4

q11 = (D/(2*tau1))*(1 - np.exp(-(2*Ts)/tau1))
q22 = (D/(2*tau1))*(1 - np.exp(-(2*Ts)/tau2))
q33 = (D/(2*tau1))*(1 - np.exp(-(2*Ts)/tau3))

kf.Q = np.array([
    [q11, 0, 0, 0, 0, 0, 0],
    [0, q22, 0, 0, 0, 0, 0],
    [0, 0, q33, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]])

kf.R =  np.array([
    [0.01, 0, 0, 0, 0, 0, 0],
    [0, 0.01, 0, 0, 0, 0, 0],
    [0, 0, 0.01, 0, 0, 0, 0],
    [0, 0, 0, 0.0001, 0, 0, 0],
    [0, 0, 0, 0, 0.0001, 0, 0],
    [0, 0, 0, 0, 0, 0.0001, 0],
    [0, 0, 0, 0, 0, 0, 0.0001]])

kf.P *= 1000